## Loading Features (with generic numpy loading function) & Parcellations (with ci_lib package)

#### Imports

In [2]:
# Loading 
from pathlib import Path
from os import path
import sys
from os import listdir
from os.path import isfile, join

# Custom library from WIPAR
sys.path.append(str((Path.cwd().parent))) #use local version of ci_lib during development
import ci_lib
from ci_lib.utils import snakemake_tools
from ci_lib.features import Features, Means, Raws, Covariances, AutoCovariances, Moup, AutoCorrelations, FeatureType
from ci_lib.plotting import graph_circle_plot, plot_glassbrain_bokeh, draw_neural_activity
from ci_lib import DecompData
from ci_lib.feature_selection import RFE_pipeline
from ci_lib.networks import construct_network, construct_network_from_feat, add_bokeh_attributes

# Processing
import numpy as np
import networkx as nx

# Dict Formating
import json     
import collections
import pandas as pd

# Plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pickle


In [3]:
#Helper functions
def print_dict_keys(print_dict,level=0):
    if type(print_dict) is dict:
        print(f"Level {level}:",end=" ")
        print(list(print_dict.keys()))
        print_dict_keys(next(iter(print_dict.values())),level+1)
    else:
        print(f"Example shape of Value (feature): {print_dict.shape}")

def dict_to_shapes(dict_):
    if type(dict_) is dict:
        dict_ = dict_.copy()
        for key,val in dict_.items():
            dict_[key] = dict_to_shapes(val)
        return dict_
    else:
        return len(dict_)

#Used to get session name on last dim
def get_key_last_dim(rec_dict):
    #Look one level ahead
    if type(rec_dict) is dict:
        arbitary_next_dict = next(iter(rec_dict.values()))
    else:
        print("not a  dict")
        print(type(rec_dict))
        return rec_dict

    if type(arbitary_next_dict) is dict:
        #If still dict continue recursion with next element
        return get_key_last_dim(arbitary_next_dict)
    else:
        #Else return a key from current level
        return list(rec_dict.keys())[0]  #TODO this will break if 'All' becomes first entry as it has no parcellation and is only the aggregation of the features from different sessions


## Loading
* Feature Values (exported, can be loaded using generic numpy loading function)
* Parcellations to get labels of nodes (custom class, load with ci_lib package)

In [4]:
import os
''' 
Example of loading exports from WIPAR:
    Load dictionary that aggregates Feature Values {Parcellation x Feature x Condition x  (Combined Sessions + Indivdual Sessions)}
'''

#Manually define export file for loading (otherwise most recent export is loaded)
overwrite_file= "feats_hashce9d60696cbe4f9c18f97cd3815bf40e.npy"

export_path = Path.cwd().parent/"results/exports/"

if overwrite_file is not None:
    #Loads manually defined export file
    loading_path = export_path/overwrite_file
else:
    #Iterates all export files to find most recent
    files = [export_path/f for f in listdir(export_path) if isfile(join(export_path, f))]
    try:
        loading_path = files[np.argmin([os.path.getmtime(f) for f in files])]

    except ValueError as err:
        if 'empty sequence' in str(err):
            print("Export folder is empty, run the feature rule first")
            loading_path = None 

#Loading
if loading_path is not None:   
    with open(loading_path, 'rb') as file:
            feat_dict = pickle.load(file)
    #feat_dict = np.load(loading_path, allow_pickle=True).tolist()
    

    print(f"Loaded feature dict from: {loading_path}")
    print("With the following structure:")
    print_dict_keys(feat_dict)





Loaded feature dict from: /home/kuehn/WIPAR/calcium-imaging-analysis/results/exports/feats_hashce9d60696cbe4f9c18f97cd3815bf40e.npy
With the following structure:
Level 0: ['LocaNMF', 'anatomical']
Level 1: ['dFC', 'FC']
Level 2: ['LR-LS', 'LR-RS', 'RR-LS', 'RR-RS']
Level 3: ['All#64d69c09']
Level 4: ['All#64d69c09']
Example shape of Value (feature): (2549, 134, 138, 138)


In [5]:
''' 
Example of loading custom classes from WIPAR:
    Loads the DecompData object for an arbirtary session for each parcellation
'''

#Get arbitrary session from last dim of feat_dict
example_session =   Path(get_key_last_dim(feat_dict)) 

#Get list of parcellations and features
parcellations = list(feat_dict.keys())

#Get feature type for arbitray parcellation
features = list(next(iter(feat_dict.values())).keys())

#Get labels for each parcellation
labels = {}
parcellation = {}

for parcellation_name in list(feat_dict.keys()):

    #Load parcellation from arbitrary session to get labels
    res_path = Path.cwd().parent /'results' /'data'
    parcellation_path = Path(res_path/example_session/Path(parcellation_name)/"data.h5")
    parcellation[parcellation_name]= DecompData.load(parcellation_path)
    print(parcellation)
    labels[parcellation_name] = parcellation[parcellation_name].spatial_labels


print(labels)

#Ignore that hashes do not match, I'll remove that as hashes of dicts/objects are just not deterministic in python

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/home/kuehn/WIPAR/calcium-imaging-analysis/results/data/All#64d69c09/LocaNMF/data.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [15]:
#Export for MOUSA BA
print(dict_to_shapes(feat_dict))

dict_MOUSA_stimulus = feat_dict["LocaNMF"]['dFC'].copy()

new_dict = None
for cond, val in dict_MOUSA_stimulus.items():
    #print(cond)
    #print(next(iter(val.values())).shape)
    if "LR" in cond:
        key = "leftResponse" 
    

        all = list(val.keys())[0]
        #new_dict[key] =  new_dict.get(key,None)
        print(val[all][all])
        new_dict= val[all][all] if new_dict is None else np.concatenate((new_dict, val[all][all]))

print(dict_to_shapes(new_dict))
#print(dict_MOUSA_stimulus)
#labels_MOUSA = labels["anatomical"]

label = "choice_dFC_LocaNMF_"+key
path = export_path / "BA_thesis"
#np.save(path / "anatomical_labels",labels_MOUSA)
(path / f"{label}.npy").touch()
np.save(path / f"{label}.npy",new_dict)


#np.load(path / "stimulus_FC.npy",allow_pickle=True)[()]
#np.load(path / "anatomical_labels.npy",allow_pickle=True)[()]
new_dict = None
for cond, val in dict_MOUSA_stimulus.items():
    #print(cond)
    #print(next(iter(val.values())).shape)
    if "RR" in cond:
        key = "rightResponse" 
    

        all = list(val.keys())[0]
        #new_dict[key] =  new_dict.get(key,None)
        #new_dict[key] = (val[all][all]) if new_dict[key] is None else np.concatenate((new_dict[key], (val[all][all]))) 
        new_dict= val[all][all] if new_dict is None else np.concatenate((new_dict, val[all][all]))

print(dict_to_shapes(new_dict))
#print(dict_MOUSA_stimulus)
#labels_MOUSA = labels["anatomical"]

label = "choice_dFC_LocaNMF_"+key
path = export_path / "BA_thesis"
#np.save(path / "anatomical_labels",labels_MOUSA)
(path / f"{label}.npy").touch()
np.save(path / f"{label}.npy",new_dict)

#elif "RR" in cond:
#        key = "rightResponse"

{'LocaNMF': {'dFC': {'LR-LS': {'All#64d69c09': {'All#64d69c09': 2549}}, 'LR-RS': {'All#64d69c09': {'All#64d69c09': 932}}, 'RR-LS': {'All#64d69c09': {'All#64d69c09': 1206}}, 'RR-RS': {'All#64d69c09': {'All#64d69c09': 2637}}}, 'FC': {'LR-LS': {'All#64d69c09': {'All#64d69c09': 2549}}, 'LR-RS': {'All#64d69c09': {'All#64d69c09': 932}}, 'RR-LS': {'All#64d69c09': {'All#64d69c09': 1206}}, 'RR-RS': {'All#64d69c09': {'All#64d69c09': 2637}}}}, 'anatomical': {'dFC': {'LR-LS': {'All#64d69c09': {'All#64d69c09': 2549}}, 'LR-RS': {'All#64d69c09': {'All#64d69c09': 932}}, 'RR-LS': {'All#64d69c09': {'All#64d69c09': 1206}}, 'RR-RS': {'All#64d69c09': {'All#64d69c09': 2637}}}, 'FC': {'LR-LS': {'All#64d69c09': {'All#64d69c09': 2549}}, 'LR-RS': {'All#64d69c09': {'All#64d69c09': 932}}, 'RR-LS': {'All#64d69c09': {'All#64d69c09': 1206}}, 'RR-RS': {'All#64d69c09': {'All#64d69c09': 2637}}}}}
[[[[ 8.65368374e-09  1.22616683e-06  1.06812450e-06 ...  1.06368301e-06
     1.05433989e-06 -1.81904176e-08]
   [ 1.22616683

In [ ]:
path = Path.cwd().parent/"results/exports/BA_thesis" #My local path of the file
choice_FC_anatomical = np.load(path / "choice_FC_fulltrial.npy",allow_pickle=True) [()] #dict that contains different conditions

print(choice_FC_anatomical.keys()) #the different conditions
print(choice_FC_anatomical["rightResponse"].shape) # the array containing all FCs for different trials from that condition with the dimension trials x timepoints x nodes x nodes  = trials x timepoints x adjacent matrix

dict_keys(['leftResponse', 'rightResponse'])
(3843, 1, 64, 64)
